Find out how many orders, how many products and how many sellers are in the data.
How many products have been sold at least once? Which is the product contained in more orders?

In [7]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .appName("Exercise1") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/19 13:16:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/19 13:16:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [16]:
sellers_table = spark.read.parquet("./sellers_parquet")

In [17]:
products_table = spark.read.parquet("./products_parquet")

In [18]:
sales_table = spark.read.parquet("./sales_parquet")

In [19]:
print('num of sellers: ', sellers_table.count())

num of sellers:  10


In [20]:
print('num of products: ', products_table.count())

num of products:  75000000


In [21]:
print('num of sales: ', sales_table.count())

num of sales:  20000040


In [23]:
sales_table.select("order_id").show(5)

+--------+
|order_id|
+--------+
|       1|
|       2|
|       3|
|       4|
|       5|
+--------+
only showing top 5 rows



In [32]:
sales_table.filter(sales_table["num_pieces_sold"] > 55).count()

9001150

In [75]:
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import col
from pyspark.sql.functions import count
from pyspark.sql.functions import avg

In [42]:
print('num of products sold at least once')

num of products sold at least once


In [43]:
sales_table.agg(countDistinct(col("product_id"))).show()

+-----------------+
|count(product_id)|
+-----------------+
|           993429|
+-----------------+



In [54]:
sales_table.groupBy(col("product_id")).agg(count("*").alias("cnt")).orderBy(col("cnt").desc()).show(10)

+----------+--------+
|product_id|     cnt|
+----------+--------+
|         0|19000000|
|  35669461|       3|
|  28592106|       3|
|  52606213|       3|
|  36269838|       3|
|  31136332|       3|
|   2316238|       3|
|  67723231|       3|
|  72017876|       3|
|  69790381|       3|
+----------+--------+
only showing top 10 rows



In [66]:
#sales_table.agg(col(sales_table.distinct())).show()
distinct_products = sales_table.select('product_id').distinct().collect()
#df.select('column1').distinct().collect()

In [69]:
len(distinct_products)

993429

How many distinct products have been sold in each day?

In [73]:
sales_table.groupBy(col("date")).agg(countDistinct(col("product_id")).alias("distinct_products_sold")).orderBy(
    col("distinct_products_sold").desc()).show()

+----------+----------------------+
|      date|distinct_products_sold|
+----------+----------------------+
|2020-07-06|                100765|
|2020-07-09|                100501|
|2020-07-01|                100337|
|2020-07-03|                100017|
|2020-07-02|                 99807|
|2020-07-05|                 99796|
|2020-07-04|                 99791|
|2020-07-07|                 99756|
|2020-07-08|                 99662|
|2020-07-10|                 98973|
+----------+----------------------+



What is the average revenue of the orders?

In [78]:
print('average num of pieces sold')

average num of pieces sold


In [77]:
sales_table.agg(avg(col("num_pieces_sold"))).show()

+--------------------+
|avg(num_pieces_sold)|
+--------------------+
|  50.498064453871095|
+--------------------+



In [80]:
# create a new column revenue_of_one_product = price * num_pieces_sold
sales_table.join(products_table, sales_table["product_id"] == products_table["product_id"], "inner")\
    .agg(avg(products_table["price"] * sales_table["num_pieces_sold"])).show()

+------------------------------+
|avg((price * num_pieces_sold))|
+------------------------------+
|            1246.1338560822878|
+------------------------------+



For each seller, what is the average % contribution of an order to the seller's daily quota?
### Example
If Seller_0 with `quota=250` has 3 orders:
Order 1: 10 products sold
Order 2: 8 products sold
Order 3: 7 products sold
The average % contribution of orders to the seller's quota would be:
Order 1: 10/105 = 0.04
Order 2: 8/105 = 0.032
Order 3: 7/105 = 0.028
Average % Contribution = (0.04+0.032+0.028)/3 = 0.03333

In [ ]:
# for each seller in sellers table find quota.
# For each seller find orders in sales_table.
# contribution = sum(num_pieces_sold) / (num_orders * daily_target)

In [86]:
sellers_table.filter(sellers_table["daily_target"] > 0).count()

10

In [90]:
print(sales_table.join(sellers_table, sales_table["seller_id"] == sellers_table["seller_id"], "inner").withColumn(
    "ratio", sales_table["num_pieces_sold"]/sellers_table["daily_target"]
).groupBy(sales_table["seller_id"]).agg(avg("ratio")).show())

22/01/19 16:13:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/19 16:13:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/19 16:13:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/19 16:13:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/19 16:13:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/19 16:13:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+---------+--------------------+
|seller_id|          avg(ratio)|
+---------+--------------------+
|        3|1.628885370565975...|
|        7|2.595228787788204...|
|        8|9.213030375409368E-5|
|        0|2.019885898974880...|
|        1|1.964233366461213...|
|        2|6.690408001064177E-5|
|        4|3.296428039825307E-5|
|        5|4.211073965904662...|
|        6|4.782147194370051E-5|
|        9|3.837913136180193...|
+---------+--------------------+

None
